In [1]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import tensorflow as tf
from tensorflow.keras import layers
import datag
import cv2
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  0


In [2]:
path_dataset = 'D:/Python/proyectoFinal/dataset/salida/'
nombres_directorios = datag.extraer_nombres(path_dataset)
print(nombres_directorios)

etiquetas = []
for index, directorio in enumerate(nombres_directorios):
    nombres_archivos = datag.extraer_nombres(path_dataset + directorio)
    total_archivos = len(nombres_archivos)
    for _ in range(total_archivos):
        etiquetas.append(index)
print(f'total etiquetas: {len(etiquetas)}')

imagenes = [] 
for directorio in nombres_directorios:
    nombres_archivos = datag.extraer_nombres(path_dataset + directorio)
    for nombre_archivo in nombres_archivos:
        img_array = cv2.imread(path_dataset + directorio + '/' + nombre_archivo, cv2.IMREAD_GRAYSCALE)
        img_array = img_array / 255 # normalizo
        imagenes.append(img_array)
print(f'total datos: {len(imagenes)}')

['Abraham_Danilo_Miranda_Lopez', 'Alejandro_Benjamin_Rocano_Lopez', 'Alex_Ignacio_Tigselema_Pacheco', 'Andres_Francisco_Garcia_Flores', 'Andres_Patricio_Tapia_Gonzalez', 'Bryan_Eduardo_Martinez_Nunez', 'Christian_Francisco_Nunez_Vaca', 'Christian_Marcelo_Jaramillo_Collazo', 'Daniel_Gerardo_Galarza_Tamayo', 'Dennis_Sebastian_Palate_Dominguez', 'Dennis_Steven_Bonilla_Granja', 'Diana_Jazmin_Pinchao_Fante', 'Diego_Fernando_Cata_Saltos', 'Diego_Mauricio_Toapanta_Llundo', 'Edison_Orlando_Lopez_Galarza', 'Ezequiel_Andres_Sanchez_Lopez', 'Fernando_Vinicio_Guevara_Bayas', 'Jeniffer_Paulina_Yaguana_Caraguay', 'Jonathan_Jose_Pazmino_Fiallos', 'Joshua_Mateo_Martinez_Urbina', 'Kevin_Jesus_Suarez_Gonzalez', 'Luis_Fernando_Zerna_Ramos', 'Marco_Farid_Ruano_Caicedo', 'Mariela_Elizabeth_Amanta_Manobanda', 'Pablo_Jhoel_Puetate_Mier', 'Silvia_Esthefania_Villacres_Chico', 'Thais_Eliana_Armijos_Troya', 'Yadira_Carolina_Allauca_Cajamarca']
total etiquetas: 18040
total datos: 18040


In [3]:
xtrain, xtest, ytrain, ytest = train_test_split(imagenes, etiquetas, random_state=10)

In [4]:
xtrain = np.array(xtrain)
ytrain = np.array(ytrain)

xtest = np.array(xtest)
ytest = np.array(ytest)

print(xtrain.shape)

(13530, 100, 100)


In [5]:
modelo = tf.keras.Sequential([
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(xtrain[0].shape[0],xtrain[0].shape[1],1)),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    
    layers.Dropout(0.5),
    layers.Flatten(),
    
    layers.Dense(64, activation=tf.nn.relu),
    layers.Dense(128, activation=tf.nn.relu),
    layers.Dense(128, activation=tf.nn.relu),
    layers.Dense(64, activation=tf.nn.relu),
    layers.Dense(len(nombres_directorios), activation=tf.nn.softmax)
])
modelo.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'])
modelo.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 98, 98, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 49, 49, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 23, 23, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 21, 21, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 10, 10, 128)       0

In [6]:
modelo.fit(xtrain, ytrain, epochs=12, batch_size=32, shuffle=True)

Epoch 1/12
423/423 [==============================] - 72s 166ms/step - loss: 2.0911 - accuracy: 0.3551
Epoch 2/12
423/423 [==============================] - 68s 161ms/step - loss: 0.8100 - accuracy: 0.7390
Epoch 3/12
423/423 [==============================] - 69s 163ms/step - loss: 0.4789 - accuracy: 0.8503
Epoch 4/12
423/423 [==============================] - 69s 162ms/step - loss: 0.3450 - accuracy: 0.8929
Epoch 5/12
423/423 [==============================] - 71s 167ms/step - loss: 0.2650 - accuracy: 0.9166
Epoch 6/12
423/423 [==============================] - 72s 171ms/step - loss: 0.2209 - accuracy: 0.9292
Epoch 7/12
423/423 [==============================] - 68s 160ms/step - loss: 0.1899 - accuracy: 0.9392
Epoch 8/12
423/423 [==============================] - 73s 172ms/step - loss: 0.1557 - accuracy: 0.9496
Epoch 9/12
423/423 [==============================] - 70s 165ms/step - loss: 0.1460 - accuracy: 0.9525
Epoch 10/12
423/423 [==============================] - 66s 156ms/step - l

In [7]:
predicciones = modelo.predict(xtest)
y_pred = np.argmax(predicciones, axis=1)

exactitud = accuracy_score(ytest, y_pred)
precision = precision_score(ytest, y_pred, average='macro')
sensibilidad = recall_score(ytest, y_pred, average='macro')

print("Exactitud:", exactitud)
print("Precisión:", precision)
print("Sensibilidad:", sensibilidad)

141/141 [==============================] - 6s 39ms/step
Exactitud: 0.9492239467849224
Precisión: 0.9452465283785827
Sensibilidad: 0.94326417760926


descgargar el modelo

In [8]:
modelo.save('modelo_cnn')
modelo.save_weights('pesos_cnn_todos.h5')

INFO:tensorflow:Assets written to: modelo_cnn\assets


INFO:tensorflow:Assets written to: modelo_cnn\assets
